In [83]:
%config IPCompleter.greedy=True
import re
import json
from sklearn.feature_extraction.text import CountVectorizer
from functools import partial

Read dataset

In [67]:
cv = CountVectorizer(token_pattern=r'\w+')
analyzer = cv.build_analyzer()

In [72]:
full_q = []
full_q_freq = []
with open('yaqq_full.tsv', 'r') as inf:
    for line in inf:
        q = ' '.join(line.strip().split()[:-1])
        full_q.append(' '.join(analyzer(q)))
        full_q_freq.append(int(line.strip().split()[-1]))

### Filtering

Filters

In [73]:
# all filters

def filter_shorter_than_n(q, n=2):
    q = ' '.join(analyzer(q))
    return len(q.split()) > n

def filter_starts_with_how(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'как':
        return True
    whitelist = [
        'зовут',
        'звали',
        'называется',
        'называли',
        'называют',
        'называлось',
        'назывались',
        'называлась',
        'назывался',
        'называются'
    ]
    if words[1] in whitelist:
        return True
    return False

def filter_starts_with_what(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'что':
        return True
    blacklist = [
        'будет',
        'нужно',
        'делать',
        'можно',
        'лучше'
    ]
    if words[1] in blacklist:
        return False
    return True

def filter_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = set([
        'купить',
        'почему',
        'онлайн',
        'сегодня',
        'зачем',
        'снится',
        'приготовить',
        'беременности',
        'видео',
        'документы',
        'тест', 
        'подарить', 
        'снятся', 
        'калорий',
        'беременным', 
        'бесплатно', 
        'отдохнуть', 
        'сниться', 
        'нельзя', 
        'заниматься', 
        'беременность', 
        'отдыхать', 
        'забеременеть', 
        'месячных', 
        'сексом', 
        'похудеть', 
        'контакте', 
        'загранпаспорт',
        'сайт',
        'может',
        'фото'
    ])
    words = q.split()
    for word in words:
        if word in stop_words:
            return False
    return True

def filter_stop_substr(q):
    q = ' '.join(analyzer(q))
    stop_substr = [
        '100 к 1',
        'сто к одному',
        'можно ли',
        'где можно'
    ]
    for sb in stop_substr:
        if sb in q:
            return False
    return True

def filter_starts_with_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = [
        'игра',
        'игры',
        'с чем',
        'чем ',
        'сколько стоит'
    ]
    for sw in stop_words:
        if q.startswith(sw):
            return False
    return True

def filter_must_start_with_question_word(q):
    ws = ' '.join(analyzer(q))
    q_words = [
        'кто ',
        'где ',
        'когда ',
        'какой ',
        'каком '
    ]
    for qw in q_words:
        if qw in ws:
            return True
    return False
    

def filter_unknown_words(q):
    ws = analyzer(q)
    for w in ws:
        if w not in russ_vocab:
            return False
    return True

Run filters

In [74]:
def apply_filters(q, filters):
    new_q = q
    for f in filters:
        new_q = list(filter(f, new_q))
    return new_q

def print_q(qs):
    for q in qs:
        print(q)

In [75]:
all_filters = [
    filter_shorter_than_n,
    filter_starts_with_how,
    filter_starts_with_what,
    filter_stop_words,
    filter_stop_substr,
    filter_starts_with_stop_words
]

filtered_q = apply_filters(
    q=full_q,
    filters=all_filters
)

In [76]:
filtered_q[:1000]

['ты кто такой давай до свидания',
 'что такое любовь',
 'гороскопы которые сбываются',
 'какой фильм посмотреть',
 'что посмотреть из фильмов',
 'о чем еще говорят мужчины',
 'что такое молочница',
 'когда же я высплюсь',
 'когда будет конец света',
 'какой летчик первым преодолел сверхзвуковой барьер в горизонтальном полете',
 'о чем говорят мужчины',
 'где этот дом',
 'что такое 228',
 'фильмы которые стоит посмотреть',
 'кто хочет стать миллионером',
 'смотри ком трансляции',
 'что где когда',
 'что такое имхо',
 'книги которые должен прочитать каждый',
 'какого смурфа мы тут делаем',
 'о чем говорят мужчины 2',
 'будет ли конец света 21 декабря 2012 года',
 'кому на руси жить хорошо',
 'куда приводят мечты',
 'стыдно когда видно',
 'сколько лет путину',
 'кто сыграл высоцкого в фильме спасибо что живой',
 'в чем встречать новый год 2013',
 'будет ли конец света в 2012 году',
 'хочу как ты',
 'что такое секс',
 'куда сходить в москве',
 'кто первым совершил беспосадочный перелет че

### Properties search

Search methods

In [77]:
def finder_substr(q, prop):
    if re.search(prop, q) is not None:
        return True
    return False

Run search

In [78]:
def apply_properties_finder(q, finder, prop):
    f = partial(finder, prop=prop)
    return list(filter(f, q))

In [79]:
def find_property(aliases):
    qs = []
    for alias in aliases:
        cur_qs = apply_properties_finder(filtered_q, finder_substr, alias)
        qs.extend(cur_qs)
    return qs

Load aliases

In [91]:
with open('properties_aliases.json', 'r') as inf:
    prop_aliases = json.load(inf)

Search

In [92]:
find_property(prop_aliases['P577'])

['когда впервые и кем уральские горы были названы своим современным именем',
 'кто и когда впервые использовал термин культурология',
 'когда впервые были анонсированы usb накопители verbatim micro на российском рынке',
 'когда впервые стали использовать наволочку',
 'когда впервые открылся рождественский базар винер адвентцаубер',
 'когда впервые перевели время в ссср',
 'кем и когда впервые получена медь',
 'когда впервые появился термин социология',
 'когда впервые и кем уральские горы были названы своим современным именем',
 'когда впервые перевели часы']